<a href="https://colab.research.google.com/github/vipashaaV321/3D-Datavisualization/blob/main/IR_GRU4RecBE_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Reshape, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
from sklearn.metrics import ndcg_score

In [ ]:
#ACCESS DATASET FROM GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
datapath= "/content/drive/MyDrive/Data/movie_rating_plot.csv"
data= pd.read_csv(datapath)

In [ ]:
data.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres,plot
0,0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
1,1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
2,2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
3,3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...
4,4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,A cowboy doll is profoundly threatened and jea...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Activation, LeakyReLU, Add
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer


# Encode user and movie IDs
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

data['userId'] = user_encoder.fit_transform(data['userId'])
data['movieId'] = movie_encoder.fit_transform(data['movieId'])

# Convert 'plot' column to strings
data['plot'] = data['plot'].astype(str)

# Tokenize and pad movie plots using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenized_plots = tokenizer(data['plot'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define model architecture
def create_gru4recbe_model():
    K = 50  # Dimensionality of the movie ID embeddings
    B = 128  # Dimensionality of the BERT plot embeddings
    D = 100  # Dimensionality of the hidden state in GRU


    # Input layers
    input_movie = Input(shape=(1,), name='movie_id_input')
    input_plot = Input(shape=(B,), name='plot_input')

    # Embedding layers
    embedding_movie = Embedding(input_dim=len(movie_encoder.classes_), output_dim=K, name='movie_embedding')(input_movie)

    # Projection layer for plot embeddings
    projected_plot = Dense(K, activation=LeakyReLU(), name='plot_projection')(input_plot)

    # Input Aggregation
    merged_input = Add(name='input_aggregation')([embedding_movie, projected_plot])

    # Gated Recurrent Unit (GRU) layer
    gru_output = GRU(D, name='gru_layer')(merged_input)

    # Decoder layer
    output_layer = Dense(1, activation='sigmoid', name='output')(gru_output)

    # Create the model
    model = Model(inputs=[input_movie, input_plot], outputs=output_layer)

    return model

# Compile the model
model = create_gru4recbe_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 movie_id_input (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 plot_input (InputLayer)     [(None, 128)]                0         []                            
                                                                                                  
 movie_embedding (Embedding  (None, 1, 50)                147250    ['movie_id_input[0][0]']      
 )                                                                                                
                                                                                            

In [ ]:
# Train the model
model.fit(
    x=[train_data['movieId'], tokenized_plots['input_ids'].numpy()],
    y=train_data['rating'],
    epochs=5,
    batch_size=16,
    validation_split=0.1
)

Epoch 1/5
2633/2633 [==============================] - 26s 8ms/step - loss: -333.4196 - accuracy: 0.0332 - val_loss: -640.0156 - val_accuracy: 0.0346
Epoch 2/5
2633/2633 [==============================] - 21s 8ms/step - loss: -931.9431 - accuracy: 0.0332 - val_loss: -1221.8967 - val_accuracy: 0.0346
Epoch 3/5
2633/2633 [==============================] - 20s 8ms/step - loss: -1513.4073 - accuracy: 0.0332 - val_loss: -1802.2686 - val_accuracy: 0.0346
Epoch 4/5
2633/2633 [==============================] - 21s 8ms/step - loss: -2094.5227 - accuracy: 0.0332 - val_loss: -2382.5444 - val_accuracy: 0.0346
Epoch 5/5
2633/2633 [==============================] - 23s 9ms/step - loss: -2675.1709 - accuracy: 0.0332 - val_loss: -2962.5955 - val_accuracy: 0.0346


In [ ]:
# Preprocess input data for prediction
user_ids = test_data['userId'].values  # Replace with the appropriate user IDs
movie_ids = test_data['movieId'].values
plots = test_data['plot'].values

user_ids_encoded = user_encoder.transform(user_ids)
movie_ids_encoded = movie_encoder.transform(movie_ids)

# Tokenize and pad movie plots using BERT tokenizer
tokenized_plots_test = tokenizer(plots.tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Make predictions
predictions = model.predict({'movie_id_input': movie_ids_encoded, 'plot_input': tokenized_plots_test['input_ids']})

# Combine user, movie, and predicted ratings into a DataFrame
results_df = pd.DataFrame({'userId': user_ids, 'movieId': movie_ids, 'predicted_rating': predictions.flatten()})

# Get top 5 movie recommendations for each user
top_5_recommendations = results_df.groupby('userId').apply(lambda x: x.nlargest(5, 'predicted_rating')).reset_index(drop=True)

print("helo")

# Display the top 5 recommendations
print(top_5_recommendations[['userId', 'movieId', 'predicted_rating']])


366/366 [==============================] - 1s 3ms/step
helo
      userId  movieId  predicted_rating
0          0      461               1.0
1          0     1616               1.0
2          0      201               1.0
3          0      781               1.0
4          0       43               1.0
...      ...      ...               ...
2653     609     2651               1.0
2654     609     2937               1.0
2655     609      461               1.0
2656     609      183               1.0
2657     609      956               1.0

[2658 rows x 3 columns]


In [ ]:
# Assuming user_id_input is the user ID provided by the user
user_id_input = 123  # Replace this with the actual user ID

# Filter recommendations for the specified user
user_recommendations = top_5_recommendations[top_5_recommendations['userId'] == user_id_input]

# Display the recommendations for the specified user
print(f"Top 5 recommendations for User {user_id_input}:")
print(user_recommendations[['userId', 'movieId', 'predicted_rating']])


In [ ]:
import numpy as np

def hit_rate(actual, predicted):
    """
    Calculate Hit Rate.

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Set of items predicted by the model.

    Returns:
    - Hit Rate.
    """
    intersection = set(actual) & set(predicted)
    return len(intersection) / len(actual)

def reciprocal_rank(actual, predicted):
    """
    Calculate Mean Reciprocal Rank (MRR).

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Ordered list of items predicted by the model.

    Returns:
    - MRR.
    """
    for i, item in enumerate(predicted, 1):
        if item in actual:
            return 1 / i
    return 0

def ndcg(actual, predicted, k=None):
    """
    Calculate Normalized Discounted Cumulative Gain (NDCG).

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Ordered list of items predicted by the model.
    - k: The maximum number of items to consider.

    Returns:
    - NDCG.
    """
    if k is None:
        k = len(predicted)

    dcg = 0
    idcg = 0

    for i in range(min(k, len(predicted))):
        item = predicted[i]
        if item in actual:
            dcg += 1 / np.log2(i + 2)
        idcg += 1 / np.log2(i + 2)

    return dcg / idcg if idcg != 0 else 0


In [ ]:
# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in top_5_recommendations.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(group['movieId'])

# Calculate metrics for each user
hit_rates = [hit_rate(actual_items[user], predicted_items[user]) for user in actual_items]
mrr = np.mean([reciprocal_rank(actual_items[user], predicted_items[user]) for user in actual_items])
ndcg_5 = np.mean([ndcg(actual_items[user], predicted_items[user], k=5) for user in actual_items])
ndcg_10 = np.mean([ndcg(actual_items[user], predicted_items[user], k=10) for user in actual_items])

print("Hit Rates:", hit_rates)
print("MRR:", mrr)
print("NDCG@5:", ndcg_5)
print("NDCG@10:", ndcg_10)

Hit Rates: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0

In [ ]:
# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in top_5_recommendations.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(group['movieId'])

    # Optional: Print for debugging
    # print(f"User {user_id} - Actual: {actual_items[user_id]}, Predicted: {predicted_items[user_id]}")

# Calculate metrics for each user
hit_rates = [hit_rate(actual_items[user], predicted_items[user]) for user in actual_items]
mrr = np.mean([reciprocal_rank(actual_items[user], predicted_items[user]) for user in actual_items])
ndcg_5 = np.mean([ndcg(actual_items[user], predicted_items[user], k=5) for user in actual_items])
ndcg_10 = np.mean([ndcg(actual_items[user], predicted_items[user], k=10) for user in actual_items])

print("Hit Rates:", hit_rates)
print("MRR:", mrr)
print("NDCG@5:", ndcg_5)
print("NDCG@10:", ndcg_10)


User 0 - Actual: {201, 43, 781, 461, 1616}, Predicted: [461, 1616, 201, 781, 43]
User 2 - Actual: {961, 565, 2941, 30, 1823}, Predicted: [2941, 1823, 961, 565, 30]
User 3 - Actual: {897, 1444, 977, 690, 819}, Predicted: [977, 690, 819, 897, 1444]
User 4 - Actual: {32, 464, 337, 52, 123}, Predicted: [52, 32, 464, 123, 337]
User 5 - Actual: {14, 238, 655, 90, 93}, Predicted: [90, 93, 655, 238, 14]
User 6 - Actual: {224, 2016, 1904, 2006, 1430}, Predicted: [1430, 224, 1904, 2006, 2016]
User 7 - Actual: {472, 504, 398, 297}, Predicted: [472, 504, 297, 398]
User 8 - Actual: {704, 329, 899, 1486}, Predicted: [329, 704, 899, 1486]
User 9 - Actual: {2144, 257, 1004, 2895, 506}, Predicted: [2895, 1004, 257, 506, 2144]
User 10 - Actual: {1188, 9, 398, 1372, 412}, Predicted: [1188, 398, 1372, 9, 412]
User 11 - Actual: {1536, 225, 2727, 315, 476}, Predicted: [2727, 1536, 315, 225, 476]
User 12 - Actual: {899, 2884, 43, 2475, 1938}, Predicted: [2884, 2475, 43, 1938, 899]
User 13 - Actual: {163, 325

In [ ]:
# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in test_data.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(top_5_recommendations.loc[top_5_recommendations['userId'] == user_id, 'movieId'])

    # Optional: Print for debugging
    print(f"User {user_id} - Actual: {actual_items[user_id]}, Predicted: {predicted_items[user_id]}")

# Calculate metrics for each user
hit_rates = [hit_rate(actual_items[user], predicted_items[user]) for user in actual_items]
mrr = np.mean([reciprocal_rank(actual_items[user], predicted_items[user]) for user in actual_items])
ndcg_5 = np.mean([ndcg(actual_items[user], predicted_items[user], k=5) for user in actual_items])
ndcg_10 = np.mean([ndcg(actual_items[user], predicted_items[user], k=10) for user in actual_items])

print("Hit Rates:", hit_rates)



User 0 - Actual: {5, 781, 783, 913, 786, 1690, 291, 1956, 1575, 43, 2604, 436, 954, 1083, 956, 828, 1598, 62, 320, 1474, 1219, 1479, 201, 2761, 1996, 461, 1616, 594, 995, 484, 2794, 1903, 1521, 632, 1145, 1916, 2429, 510}, Predicted: [461, 1616, 201, 781, 43]
User 2 - Actual: {961, 565, 831, 2941, 30, 1823}, Predicted: [2941, 1823, 961, 565, 30]
User 3 - Actual: {897, 2306, 135, 2827, 783, 1423, 1297, 2323, 278, 2077, 1182, 31, 1570, 2083, 1444, 676, 680, 686, 816, 690, 819, 694, 696, 701, 702, 326, 2507, 977, 211, 1752, 2523, 1639, 2411, 2035}, Predicted: [977, 690, 819, 897, 1444]
User 4 - Actual: {32, 260, 198, 398, 251, 464, 337, 52, 123, 413}, Predicted: [52, 32, 464, 123, 337]
User 5 - Actual: {257, 2, 3, 514, 5, 140, 14, 655, 398, 401, 20, 277, 407, 25, 543, 801, 418, 422, 298, 555, 300, 815, 306, 180, 308, 55, 568, 444, 192, 450, 325, 71, 455, 77, 209, 594, 340, 212, 470, 469, 90, 220, 93, 221, 229, 232, 236, 238, 124, 115, 630, 760, 506, 379, 252, 639}, Predicted: [90, 93, 655

In [ ]:
def precision_at_k(actual, predicted, k):
    """
    Calculate Precision@k.

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Ordered list of items predicted by the model.
    - k: The number of top items to consider.

    Returns:
    - Precision@k.
    """
    intersection = set(predicted[:k]) & set(actual)
    return len(intersection) / k if k != 0 else 0

def recall_at_k(actual, predicted, k):
    """
    Calculate Recall@k.

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Ordered list of items predicted by the model.
    - k: The number of top items to consider.

    Returns:
    - Recall@k.
    """
    intersection = set(predicted[:k]) & set(actual)
    return len(intersection) / len(actual) if len(actual) != 0 else 0

# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in test_data.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(top_5_recommendations.loc[top_5_recommendations['userId'] == user_id, 'movieId'])

# Calculate Precision@k and Recall@k for each user
k_values = [1, 5, 10]  # You can modify this list as needed

precision_at_k_values = {}
recall_at_k_values = {}

for k in k_values:
    precision_at_k_values[k] = np.mean([precision_at_k(actual_items[user], predicted_items[user], k) for user in actual_items])
    recall_at_k_values[k] = np.mean([recall_at_k(actual_items[user], predicted_items[user], k) for user in actual_items])

# Print Precision@k and Recall@k values for each k
for k in k_values:
    print(f"Precision@{k}: {precision_at_k_values[k]:.4f}")
    print(f"Recall@{k}: {recall_at_k_values[k]:.4f}")


Precision@1: 1.0000
Recall@1: 0.1786
Precision@5: 0.8919
Recall@5: 0.5573
Precision@10: 0.4460
Recall@10: 0.5573


In [ ]:
mrr = np.mean([reciprocal_rank(actual_items[user], predicted_items[user]) for user in actual_items])
ndcg_5 = np.mean([ndcg(actual_items[user], predicted_items[user], k=5) for user in actual_items])
ndcg_10 = np.mean([ndcg(actual_items[user], predicted_items[user], k=10) for user in actual_items])


In [ ]:
# Example post-processing to diversify recommendations
def diversify_recommendations(predictions, diversity_factor=0.5):
    unique_predictions = set(predictions)
    num_additional_items = int(len(predictions) * diversity_factor)

    # Add diverse items to the recommendations
    diverse_recommendations = list(unique_predictions)

    # Ensure the final list has the desired length
    if len(diverse_recommendations) < len(predictions):
        diverse_recommendations.extend(list(unique_predictions)[:num_additional_items])

    return diverse_recommendations

# Usage
predicted_items[user_id] = diversify_recommendations(predicted_items[user_id])


In [ ]:
predicted_items

{0: [461, 1616, 201, 781, 43],
 2: [2941, 1823, 961, 565, 30],
 3: [977, 690, 819, 897, 1444],
 4: [52, 32, 464, 123, 337],
 5: [90, 93, 655, 238, 14],
 6: [1430, 224, 1904, 2006, 2016],
 7: [472, 504, 297, 398],
 8: [329, 704, 899, 1486],
 9: [2895, 1004, 257, 506, 2144],
 10: [1188, 398, 1372, 9, 412],
 11: [2727, 1536, 315, 225, 476],
 12: [2884, 2475, 43, 1938, 899],
 13: [297, 472, 314, 163, 325],
 14: [2224, 131, 1548, 1485, 220],
 15: [172, 914, 599, 898, 1825],
 16: [837, 46, 933, 922, 0],
 17: [506, 695, 833, 385, 2460],
 18: [2449, 1516, 376, 2637, 358],
 19: [1598, 1710, 12, 1576, 1562],
 20: [2887, 2755, 1052, 2353, 1486],
 21: [2802, 217, 2263, 1525, 277],
 22: [602, 2190, 474, 2281, 254],
 23: [963, 1804, 484, 1052, 1321],
 24: [1938],
 25: [32, 509, 314, 398],
 26: [2592, 2377, 985, 1556, 1705],
 27: [2456, 2221, 5, 1374, 2013],
 28: [908, 1324, 899, 2156, 1065],
 29: [224, 910, 793, 897, 901],
 30: [815, 9, 694, 297, 808],
 31: [658, 819, 793, 840, 319],
 32: [225, 32, 

In [ ]:
# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in top_5_recommendations.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(group['movieId'])


# Check for data leakage
train_users = set(train_data['userId'])
test_users = set(test_data['userId'])

intersection_users = train_users.intersection(test_users)

mrr = np.mean([reciprocal_rank(actual_items[user], predicted_items[user]) for user in actual_items])

print("MRR:", mrr)

MRR: 0.7999999999999999


In [ ]:
from sklearn.metrics import ndcg_score

# Assuming actual_items and predicted_items are defined
# ...

# Convert the dictionary to a list of lists
actual_list = [list(actual_items[user]) for user in actual_items]
predicted_list = [list(predicted_items[user]) for user in actual_items]

# Flatten the lists
flat_actual = [item for sublist in actual_list for item in sublist]
flat_predicted = [item for sublist in predicted_list for item in sublist]

# Calculate NDCG using sklearn
ndcg_sklearn = ndcg_score([flat_actual], [flat_predicted], k=k_values[-1])

# Print NDCG using sklearn
print(f"NDCG@{k_values[-1]} (sklearn): {ndcg_sklearn:.4f}")


NDCG@10 (sklearn): 0.5516


In [ ]:
from sklearn.metrics import ndcg_score

# Assuming actual_items and predicted_items are defined
# ...

# Convert the dictionary to a list of lists
actual_list = [list(actual_items[user]) for user in actual_items]
predicted_list = [list(predicted_items[user]) for user in actual_items]

# Flatten the lists
flat_actual = [item for sublist in actual_list for item in sublist]
flat_predicted = [item for sublist in predicted_list for item in sublist]

# Calculate NDCG using sklearn for k = 1, 5, and 10
k_values = [1, 5, 10]
for k in k_values:
    ndcg_sklearn = ndcg_score([flat_actual], [flat_predicted], k=k)
    print(f"NDCG@{k} (sklearn): {ndcg_sklearn:.4f}")


NDCG@1 (sklearn): 1.0000
NDCG@5 (sklearn): 0.6133
NDCG@10 (sklearn): 0.5516


In [ ]:
# Extract actual and predicted items for each user
actual_items = {}
predicted_items = {}

for user_id, group in test_data.groupby('userId'):
    actual_items[user_id] = set(group['movieId'])
    predicted_items[user_id] = list(top_5_recommendations.loc[top_5_recommendations['userId'] == user_id, 'movieId'])


# Calculate metrics for each user
hit_rates = [hit_rate(actual_items[user], predicted_items[user]) for user in actual_items]

print("Hit Rates for 5 Items:", hit_rates)

print(np.mean(hit_rates))

Hit Rates for 5 Items: [0.13157894736842105, 0.8333333333333334, 0.14705882352941177, 0.5, 0.08928571428571429, 0.45454545454545453, 1.0, 1.0, 0.625, 0.29411764705882354, 0.7142857142857143, 0.5555555555555556, 0.8333333333333334, 0.45454545454545453, 0.2777777777777778, 0.2777777777777778, 0.14705882352941177, 0.03164556962025317, 0.16666666666666666, 0.16666666666666666, 0.5555555555555556, 0.20833333333333334, 0.5555555555555556, 1.0, 1.0, 0.2631578947368421, 0.08620689655172414, 0.38461538461538464, 1.0, 0.4166666666666667, 0.21739130434782608, 0.21739130434782608, 1.0, 0.7142857142857143, 0.5, 1.0, 0.3333333333333333, 0.23809523809523808, 0.29411764705882354, 0.29411764705882354, 0.06578947368421052, 0.2, 0.5, 0.07142857142857142, 1.0, 0.4166666666666667, 1.0, 1.0, 0.29411764705882354, 0.09433962264150944, 0.8333333333333334, 1.0, 0.8333333333333334, 1.0, 0.625, 0.052083333333333336, 0.1724137931034483, 0.20833333333333334, 1.0, 1.0, 0.35714285714285715, 0.25, 0.07575757575757576,

In [ ]:
def hit_rate_at_k(actual, predicted, k):
    """
    Calculate Hit Rate@k.

    Parameters:
    - actual: Set of actual items the user interacted with.
    - predicted: Ordered list of items predicted by the model.
    - k: The number of top items to consider.

    Returns:
    - Hit Rate@k.
    """
    intersection = set(actual) & set(predicted[:k])
    return len(intersection) / min(k, len(actual))


In [ ]:
# Calculate Hit Rate@k for each k
k_values = [ 10]
hit_rate_at_k_values = {}

for k in k_values:
    hit_rate_at_k_values[k] = np.mean([hit_rate_at_k(actual_items[user], predicted_items[user], k) for user in actual_items])

# Print Hit Rate@k values for each k
for k in k_values:
    print(f"Hit Rate@{k}: {hit_rate_at_k_values[k]:.4f}")



Hit Rate@10: 0.6857
